# Project N°1 
Make by:

Jose Luis Hincapie Bucheli - 2125340

Carlos Andrés Hernández - 2125653

## Require Libraries

In [134]:
%%writefile requirements.txt
numpy==1.25.2
prettytable
sympy==1.12

Overwriting requirements.txt


In [135]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Discretization of the initial problem.

In [136]:
import numpy as np
#from auxiliares import limpiar_pantalla
from prettytable import PrettyTable 

# Se define la malla a tratar con las condiciones iniciales

v_0 = 1 # Velocidad inicial en x

mallax = np.array(
    [[1,  0,   0,   0,   0,   0,   0,  0],
    [1, "w1", "w2", 0, 0, "w3", "w4", 0],
    [1, "w5", "w6", 0, 0, "w7", "w8", 0],
    [1, "w9", "w10", "w11", "w12", "w13", "w14", 0],
    [1, "w15", "w16", 0, 0, "w17", "w18", 0],
    [1, "w19", "w20", 0, 0, "w21", "w22", 0],
    [1,  0,   0,   0,   0,   0,   0,  0]])

mallay = np.array(
    [[0,  0,   0,   0,   0,   0,   0,  0],
    [0, "w1", "w2", 0, 0, "w3", "w4", 0],
    [0, "w5", "w6", 0, 0, "w7", "w8", 0],
    [0, "w9", "w10", "w11", "w12", "w13", "w14", 0],
    [0, "w15", "w16", 0, 0, "w17", "w18", 0],
    [0, "w19", "w20", 0, 0, "w21", "w22", 0],
    [0,  0,   0,   0,   0,   0,   0,  0]])


r_matrix = np.zeros((22, 23), dtype=int) # Almacenar los resultados finales de la discretización

# Funtion that reshape results in a finish matrix the last colum are the numeric values

def fixMatrix(values, count):
    global r_matrix
    acum = 0
    for value in values:
        if value[0] == 'w':
            match np.where(values == value)[0][0]:
                case 0:
                    r_matrix[count][int(value[1:])-1] = 1
                case 1:
                    r_matrix[count][int(value[1:])-1] = -8
                case 2:
                    r_matrix[count][int(value[1:])-1] = 3
                case 3:
                    r_matrix[count][int(value[1:])-1] = 3
                case 4:
                    r_matrix[count][int(value[1:])-1] = 1
        else:
            match np.where(values == value)[0][0]:
                case 0:
                    acum += int(value)*1
                case 1:
                    acum += int(value)*-8
                case 2:
                    acum += int(value)*3
                case 3:
                    acum += int(value)*3
                case 4:
                    acum += int(value)*1
    r_matrix[count][-1] = acum


# Funtion that find the positions needed in navier-stokes using Finite differences
def discretization(malla):
    count = 0
    for row in range(7):
        for colum in range(8):
            if malla[row][colum][0] == 'w':
                temp = np.array([malla[row][colum+1], malla[row][colum],
                                malla[row][colum-1], malla[row-1][colum], malla[row+1][colum]])
                fixMatrix(temp, count)
                count += 1
                

if __name__ == "__main__":
    discretization(mallax)

    column_names = [f"W{i}" for i in range(1, 24)]
    column_names[-1] = "T.I"
    
    # Create a table to format de r_matrix table
    table = PrettyTable()
    table.field_names = [""] + column_names 
    
    for i, row in enumerate(r_matrix, start=1):
        equation_label = f"Ecuación {i}"
        row_as_strings = [str(value) for value in row]
        table.add_row([equation_label] + row_as_strings)
        

    print(table)
    


+-------------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|             | W1 | W2 | W3 | W4 | W5 | W6 | W7 | W8 | W9 | W10 | W11 | W12 | W13 | W14 | W15 | W16 | W17 | W18 | W19 | W20 | W21 | W22 | T.I |
+-------------+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|  Ecuación 1 | -8 | 1  | 0  | 0  | 1  | 0  | 0  | 0  | 0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  3  |
|  Ecuación 2 | 3  | -8 | 0  | 0  | 0  | 1  | 0  | 0  | 0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |
|  Ecuación 3 | 0  | 0  | -8 | 1  | 0  | 0  | 1  | 0  | 0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |
|  Ecuación 4 | 0  | 0  | 3  | -8 | 0  | 0  | 0  | 1  | 0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0  |  0 

# Second Part Of The Solution


To solve the problem of finding the solution for the parameter variables, use Successive Over-Relaxation (SOR) to aproch the solution faster.

- First check if the matrix is strictly diagonally dominant

In [5]:
'''
Funtion that checks if a matrix is diagonally dominant
X = Matrix
'''
def dd(X):
    M = np.delete(X,-1,1)
    D = np.diag(np.abs(M)) # Find diagonal coefficients
    S = np.sum(np.abs(M), axis=1) - D # Find row sum without diagonal
    if np.all(D >= S):
        print( 'matrix is diagonally dominant')
    else:
        print ('NOT diagonally dominant')

dd(r_matrix)

matrix is diagonally dominant


Now that the conditions are meet and we know the method will converge in a solution we use SOR


In [6]:
'''
Funtion that use jacobi method in a especific row
R = Row to apply jacobi
S = array of initial solutions for the row
nn = interation value, in other words for which variable we want to apply jacobi.
'''
def jacobi(R,S,nn):
    acum =0
    c = -R[-1]/R[nn]
   # print(f"{-R[-1]}/{R[nn]} = {c}")
    acum += c
    for i in range(len(R)-1):
        #print(f"{-R[i]} / {R[nn]}*{S[i]} = {(-R[i]/ R[nn]) * S[i]}")
        acum+= ( (-R[i]/ R[nn]) * S[i] ) if i != nn else 0
    return acum

In [7]:
jacobi(r_matrix[-1],np.zeros(22),21)

0.0

In [8]:
'''
Funtion that checks if the given solutions satisfy the tolerance
S = Preview solution
NS = New Solution
tol = tolerance
'''
def check_tolerance(S,NS,tol):
    #Infinity Norm
    error = ( np.max(np.abs(NS)) - np.max(np.abs(S)) )/ np.max(np.abs(NS))
    print("Error: ",error)
    if error > tol : return True 
    else: return False 
    

In [9]:
'''
Funtion that use Successive Over-Relaxation in an especific matrix.
M = Matrix of coeficients with idependent vector  
omega = stride for faster convergence.
tol = tolerance
'''

def SOR(M,omega,tol):
    l = len(M)
    S= np.zeros(l)
    OS = np.copy(S)
    acum = 0
    for i in range(l):
        c = -M[i][-1]/M[i][i]
        S[i] = c    
    while check_tolerance(OS,S,tol):
        acum += 1
        OS = np.copy(S)
        for i in range(l):
            S[i] = (1 - omega) * S[i] + omega * jacobi(M[i],S,i)
    print(f"Number of iterations: {acum}")
    return S

In [10]:
lineal_solution = SOR(r_matrix,1.2,0.1)
print("---------------")
print(lineal_solution)

Error:  1.0
Error:  0.5012888566111955
Error:  0.12072251780308257
Error:  0.004822101940646059
Number of iterations: 3
---------------
[0.49391285 0.23860971 0.00099771 0.0013469  0.71613287 0.43120175
 0.00612231 0.00512097 0.82449689 0.57724476 0.22739601 0.09039226
 0.04100276 0.01911735 0.85932119 0.59997487 0.01832543 0.01500941
 0.75815391 0.50843105 0.00782167 0.00863033]


## Conjugate Gradient Method

Applying this method allow to find the gradient, with shows the direction for which the funtions increase or decrease.

To use this method the matrix must be symmetric and positive

In [19]:
r_matrix_ni = np.delete(r_matrix,-1,1)

def symmetric(M):
    Mt = np.transpose(M) #Transpose matrix
    #print(np.equal(M,Mt)) #shows which elements of are not equal to the intial matrix (M)
    return np.array_equal(M,Mt)

In [21]:
print(f"It's the matrix symmetric? {symmetric(r_matrix_ni)}")

It's the matrix symmetric? False


In [22]:
def possitive(M):
    return np.all(np.linalg.eigvals(M)>0)

In [24]:
print(f"It's the matrix symmetric? {possitive(r_matrix_ni)}")

It's the matrix symmetric? False


Since the conditions are not met, we can't use Conjugate Gradient

----

# Newton Raphson Metohd for non-linear systems

Generating the system of nonlinear equations

In [204]:

# We want to represent nonlinear equations, which using Navier-Stokes would be of the form: 
# 2Vx(i+1,j) + 2Vx(i-1,j) + 2Vx(i,j+1 + 2Vx(i,j-1 - 8Vx(i,j - Vx(i,j)Vx(i+1,j + Vx(i,j)Vx(i-1,j - Vx(i,j)Vx(i,j+1 + Vx(i,j)Vx(i,j-1) = 0

from sympy import Symbol
import numpy as np

mallax = np.array(
    [[1,  0,   0,   0,   0,   0,   0,  0],
    [1, "w1", "w2", 0, 0, "w3", "w4", 0],
    [1, "w5", "w6", 0, 0, "w7", "w8", 0],
    [1, "w9", "w10", "w11", "w12", "w13", "w14", 0],
    [1, "w15", "w16", 0, 0, "w17", "w18", 0],
    [1, "w19", "w20", 0, 0, "w21", "w22", 0],
    [1,  0,   0,   0,   0,   0,   0,  0]])

mallay = np.array(
    [[0,  0,   0,   0,   0,   0,   0,  0],
    [0, "w1", "w2", 0, 0, "w3", "w4", 0],
    [0, "w5", "w6", 0, 0, "w7", "w8", 0],
    [0, "w9", "w10", "w11", "w12", "w13", "w14", 0],
    [0, "w15", "w16", 0, 0, "w17", "w18", 0],
    [0, "w19", "w20", 0, 0, "w21", "w22", 0],
    [0,  0,   0,   0,   0,   0,   0,  0]])

# These lists will be used to store the nonlinear equations and corresponding symbolic variables.
equations_x = []
vars_x = []

equations_y = []
vars_y = []

# This function takes a position (i, j) in the matrix grid and returns a symbol if the entry starts with "w" (indicating a variable) or an integer if not.
def determinate(i, j, grid):
    if grid[i, j][0] == "w":
        # variable
        return Symbol(grid[i, j])
    else:
        # number
        return int(grid[i, j])

# This function generates nonlinear equations based on the elements of the grid matrix. It iterates through the rows and columns of the matrix, and for each non-zero element, constructs a nonlinear equation and adds both the equation and the symbolic variable to the corresponding lists.
def generate_nonlinear_equations(grid,equations,vars):
    for i in range(1, 6):
        for j in range(1, 7):
            # Construct the nonlinear equation
            value = determinate(i, j, grid)
            if value != 0:
                equation = 2 * determinate(i + 1, j, grid) + 2 * determinate(i - 1, j, grid) + 2 * determinate(i, j + 1, grid) + 2 * determinate(i, j - 1, grid) - 8 * determinate(i, j, grid) - determinate(i, j, grid) * determinate(i + 1, j, grid) + determinate(i, j, grid) * determinate(i - 1, j, grid) - determinate(i, j, grid) * determinate(i, j + 1, grid) + determinate(i, j, grid) * determinate(i, j - 1, grid)

                equations.append(equation)
                vars.append(value)

# Call the function with the grid matrix
generate_nonlinear_equations(mallax,equations_x,vars_x)
generate_nonlinear_equations(mallay,equations_y,vars_y)



In [211]:
# Print the nonlinear equations and corresponding variables
table_x = PrettyTable()
table_x.field_names = ['Equation']
for eq in zip(equations_x):
    table_x.add_row([str(eq)])
print("Nonlinear Equations for Vx:")
print(table_x)

Nonlinear Equations for Vx:
+--------------------------------------------------------------------------------+
|                                    Equation                                    |
+--------------------------------------------------------------------------------+
|                   (-w1*w2 - w1*w5 - 7*w1 + 2*w2 + 2*w5 + 2,)                   |
|                     (w1*w2 + 2*w1 - w2*w6 - 8*w2 + 2*w6,)                      |
|                     (-w3*w4 - w3*w7 - 8*w3 + 2*w4 + 2*w7,)                     |
|                     (w3*w4 + 2*w3 - w4*w8 - 8*w4 + 2*w8,)                      |
|            (w1*w5 + 2*w1 - w5*w6 - w5*w9 - 7*w5 + 2*w6 + 2*w9 + 2,)            |
|            (-w10*w6 + 2*w10 + w2*w6 + 2*w2 + w5*w6 + 2*w5 - 8*w6,)             |
|            (-w13*w7 + 2*w13 + w3*w7 + 2*w3 - w7*w8 - 8*w7 + 2*w8,)             |
|            (-w14*w8 + 2*w14 + w4*w8 + 2*w4 + w7*w8 + 2*w7 - 8*w8,)             |
|         (-w10*w9 + 2*w10 - w15*w9 + 2*w15 + w5*w9 + 2*w5 

In [212]:
print(vars_x)

[w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14, w15, w16, w17, w18, w19, w20, w21, w22]


In [213]:
table_y = PrettyTable()
table_y.field_names = ['Equation']
for eq in zip(equations_y):
    table_y.add_row([str(eq)])
print("Nonlinear Equations for Vy:")
print(table_y)

Nonlinear Equations for Vy:
+--------------------------------------------------------------------------------+
|                                    Equation                                    |
+--------------------------------------------------------------------------------+
|                     (-w1*w2 - w1*w5 - 8*w1 + 2*w2 + 2*w5,)                     |
|                     (w1*w2 + 2*w1 - w2*w6 - 8*w2 + 2*w6,)                      |
|                     (-w3*w4 - w3*w7 - 8*w3 + 2*w4 + 2*w7,)                     |
|                     (w3*w4 + 2*w3 - w4*w8 - 8*w4 + 2*w8,)                      |
|              (w1*w5 + 2*w1 - w5*w6 - w5*w9 - 8*w5 + 2*w6 + 2*w9,)              |
|            (-w10*w6 + 2*w10 + w2*w6 + 2*w2 + w5*w6 + 2*w5 - 8*w6,)             |
|            (-w13*w7 + 2*w13 + w3*w7 + 2*w3 - w7*w8 - 8*w7 + 2*w8,)             |
|            (-w14*w8 + 2*w14 + w4*w8 + 2*w4 + w7*w8 + 2*w7 - 8*w8,)             |
|           (-w10*w9 + 2*w10 - w15*w9 + 2*w15 + w5*w9 + 2*w

In [232]:
print(vars_y)

[w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14, w15, w16, w17, w18, w19, w20, w21, w22]


Calculate the jacobian matrix for the system of nonlinear equations

In [139]:
# Define a function to calculate the Jacobian matrix for a system of nonlinear equations.

def jacobian(equations, variables):
    # Get the number of equations and variables
    rows = len(equations)
    cols = len(variables)

    # Initialize the Jacobian matrix with zeros
    J = [[0] * cols for _ in range(rows)]

    # Iterate through each equation and variable to calculate partial derivatives
    for i, eq in enumerate(equations):
        for j, var in enumerate(variables):
            # Calculate the partial derivative of the equation with respect to the variable
            J[i][j] = diff(eq, var)

    # Return the calculated Jacobian matrix
    return J


In [146]:
# x_(k+1) = x_k - (J(x_k))^-1 * f(x_k)
#  J(x_k) = (dF(x_k)/dx_k))

# (J(x_k))*x_(k+1) = (J(x_k))*x_k - f(x_k) => Without Jacobian Inverse

# Define symbolic variables
x1, x2 = symbols('x1 x2')

# Define the equations
f1 = x1**2 - x2**2 + 2*x2
f2 = 2*x1 + x2**2 - 6

# Define the Jacobian
J = jacobian([f1, f2], [x1, x2])

# Display the Jacobian
print("Jacobian:")
print(J)

Jacobian:
[[2*x1, 2 - 2*x2], [2, 2*x2]]


Evaluate symbolic functions

In [140]:
# Evaluate a symbolic function for a given set of variables and values.

def evalFunction(function, vars, values):
    # Initialize the result with the given function
    result = function

    # Number of variables
    numVars = len(vars)

    # Iterate through each variable and substitute its value in the function
    for i in range(numVars):
        result = result.subs(vars[i], values[i])

    # Convert the result to a float and return
    return float(str(result))


In [148]:
result = evalFunction(equations_x[0], vars_x, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])
print(result)

2.0


In [207]:
result = evalFunction(equations_y[0], vars_y, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])
print(result)

-1.0


In [149]:
# Evaluate a list of symbolic functions for a given set of variables and values.

def evalFunctionsAndValues(functions, vars, values):
    # List to store the results of evaluated functions
    results = []

    # Number of variables
    numVars = len(vars)

    # Iterate through each function in the list
    for func in functions:
        # Initialize the result with the current function
        result = func
        
        # Iterate through each variable and substitute its value in the function
        for i in range(numVars):
            result = result.subs(vars[i], values[i])
        
        # Append the evaluated result to the list
        results.append(float(str(result)))

    # Return the list of evaluated function results
    return results


In [150]:
functions_evaluated = evalFunctionsAndValues(equations_x, vars_x, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22])
print(functions_evaluated)


[2.0, -12.0, -35.0, -30.0, -71.0, -32.0, -134.0, -38.0, -181.0, -116.0, -66.0, -72.0, -160.0, 8.0, -405.0, 42.0, -474.0, 124.0, -156.0, 610.0, -195.0, 760.0]


Evaluate the jacobian matrix

In [238]:
# Evaluate the Jacobian matrix of a system of equations at a given point.

def evaluateJacobian(jacobian_matrix, symbols, numerical_values):
    # List to store the evaluated Jacobian matrix
    evaluated_jacobian = []
    # Iterate through each row in the Jacobian matrix
    for row in jacobian_matrix:
        # Evaluate each partial derivative in the row using given values
        row_evaluated = evalFunctionsAndValues(row, symbols, numerical_values)
        
        # Append the row of evaluated partial derivatives to the result
        evaluated_jacobian.append(row_evaluated)

    # Return the complete evaluated Jacobian matrix
    return evaluated_jacobian


Determine whether the Newton-Raphson method should stop at a given iteration, based on the evaluation of the equations of the system at a candidate solutionº.

In [153]:
def shouldStop(equations, variables, current_solution, tolerance):
    # Evaluate the system of equations at the current solution
    evaluations = np.array(evalFunctions(equations, variables, current_solution))

    # Calculate the norm (magnitude) of the vector of equation evaluations
    norm = np.linalg.norm(evaluations)

    # Check if the norm is below the specified tolerance
    if norm < tolerance:
        # If the norm is below tolerance, consider the solution accurate enough
        return True
    else:
        # If the norm is not below tolerance, more iterations are needed
        return False


In [154]:
# Define variables and equations
x, y = symbols('x y')
equations = [x**2 + y**2 - 25, x + y - 7]

# Initial guess for the solution
initial_solution = [1, 6]

# Tolerance for stopping criteria
tolerance = 1e-6

# Check if the solution is accurate enough
result = shouldStop(equations, [x, y], initial_solution, tolerance)

print(result)


False


Newton-Raphson method

In [236]:
tolerance = 1e-3
initAproximationVX = [0]*len(vars_x)
initAproximationVY = [0]*len(vars_y)

[w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, w14, w15, w16, w17, w18, w19, w20, w21, w22]


In [202]:
def newtonRaphson(system_equations, variables, initial_approximation, tolerance):
    # Initialize the solution with the initial approximation
    current_solution = initial_approximation
    
    # Calculate the Jacobian matrix of the system
    jacobian_matrix = jacobian(system_equations, variables)
    
    # Initialize the iteration counter
    iterations = 0

    while True:
        # Evaluate the Jacobian matrix at the current solution
        jacobian_evaluated = np.array(evaluateJacobian(jacobian_matrix, variables, current_solution))
        
        # Calculate the inverse of the Jacobian matrix
        jacobian_inv = np.linalg.inv(jacobian_evaluated)

        # Evaluate the system functions at the current solution
        system_evaluations = np.array(evalFunctionsAndValues(system_equations, variables, current_solution))

        # Calculate the next approximation using the Newton-Raphson method
        # x_(k+1) = x_k - (J(x_k))^-1 * f(x_k)
        next_solution = np.array(current_solution) - (jacobian_inv.dot(system_evaluations))

        # Check the convergence conditions
        if iterations >= 100 or shouldStop(system_equations, variables, next_solution, tolerance):
            return next_solution
        else:
            # Update the current solution with the new approximation
            current_solution = next_solution

        # Increment the iteration counter
        iterations += 1


In [239]:
solution_vx = newtonRaphson(equations_x, vars_x, initAproximationVX,tolerance)

solution_vy = newtonRaphson(equations_y, vars_y, initAproximationVY,tolerance)

In [240]:

# Create a table to display the solutions
table = PrettyTable()
table.field_names = ['Variable', 'Solution']

# Add rows to the table with each variable and its solution
for var, sol in zip(vars_x, solution_vx):
    table.add_row([str(var), sol])

# Print the table
print("Solution for Vx:")
print(table)

Solution for Vx:
+----------+-----------------------+
| Variable |        Solution       |
+----------+-----------------------+
|    w1    |   0.4800783093023615  |
|    w2    |  0.21266834401399803  |
|    w3    | 0.0011688907451201764 |
|    w4    |  0.000841288135277563 |
|    w5    |   0.6824740013922657  |
|    w6    |  0.35756839289724873  |
|    w7    |  0.003837009054196455 |
|    w8    |  0.002196694135354569 |
|    w9    |   0.7729767160071275  |
|   w10    |  0.45675441437112196  |
|   w11    |  0.13003793768806357  |
|   w12    |  0.03604312269242103  |
|   w13    |  0.012007459545662548 |
|   w14    |  0.004107852805995208 |
|   w15    |   0.7806708220407035  |
|   w16    |  0.43796197421814237  |
|   w17    |   0.0038502021148202  |
|   w18    | 0.0022026072517890662 |
|   w19    |   0.6407493717809911  |
|   w20    |   0.3117082382225878  |
|   w21    | 0.0011741266439800465 |
|   w22    | 0.0008445399472397637 |
+----------+-----------------------+


In [241]:

# Create a table to display the solutions
tabley = PrettyTable()
tabley.field_names = ['Variable_Y', 'Solution']

# Add rows to the table with each variable and its solution
for var, sol in zip(vars_y, solution_vy):
    tabley.add_row([str(var), sol])

# Print the table
print("Solution for Vy:")
print(tabley)


Solution for Vy:
+------------+----------+
| Variable_Y | Solution |
+------------+----------+
|     w1     |   0.0    |
|     w2     |   0.0    |
|     w3     |   0.0    |
|     w4     |   0.0    |
|     w5     |   0.0    |
|     w6     |   0.0    |
|     w7     |   0.0    |
|     w8     |   0.0    |
|     w9     |   0.0    |
|    w10     |   0.0    |
|    w11     |   0.0    |
|    w12     |   0.0    |
|    w13     |   0.0    |
|    w14     |   0.0    |
|    w15     |   0.0    |
|    w16     |   0.0    |
|    w17     |   0.0    |
|    w18     |   0.0    |
|    w19     |   0.0    |
|    w20     |   0.0    |
|    w21     |   0.0    |
|    w22     |   0.0    |
+------------+----------+
